In [5]:
import numpy as np
import os
from math import pi, sin, log
from scipy.special import dawsn

from Sire.Tools import Parameter, resolveParameters
#from Sire.Units import *
from Sire.Tools.OpenMMMD import *
# Constants
v0 = 1660.53907 # A^3, the standard state volume
R = 0.00198720425864083 # kcal mol-1, the molar gas constant

In [6]:

    # Get Cartesian restraint dict in dict form
    #cart_dict = {"anchor_points":{"l1":21, "l2":11, "l3":2, "r1":4946, "r2":4944, "r3":4949}, "equilibrium_values":{ "xr_l1_0":-1.61, "yr_l1_0": 4.48, "zr_l1_0": 0.68}, "force_constants":{"k_xr_l1":2.65, "k_yr_l1":11.91, "k_zr_l1":4.14,"k_phi":21.01, "k_theta":21.81, "k_psi":25.24}, "reference_frame_rotation":{"xl_ref":{"xl_ref_xl": -0.0255, "xl_ref_yl":0.0346, "xl_ref_zl":0.9991}, "yl_ref":{"yl_ref_xl": 0.6906, "yl_ref_yl":-0.7220, "yl_ref_zl":0.0426}, "zl_ref":{"zl_ref_xl": 0.7228, "zl_ref_yl":0.6911, "zl_ref_zl":-0.0055}}}
    cart_dict = {"anchor_points":{"l1":21, "l2":11, "l3":2, "r1":4946, "r2":4944, "r3":4949}, "equilibrium_values":{ "xr_l1_0":-1.61, "yr_l1_0": 4.48, "zr_l1_0": 0.68}, "force_constants":{"k_xr_l1":2.65, "k_yr_l1":11.91, "k_zr_l1":4.14,"k_phi":0, "k_theta":0, "k_psi":0}, "reference_frame_rotation":{"xl_ref":{"xl_ref_xl": -0.0255, "xl_ref_yl":0.0346, "xl_ref_zl":0.9991}, "yl_ref":{"yl_ref_xl": 0.6906, "yl_ref_yl":-0.7220, "yl_ref_zl":0.0426}, "zl_ref":{"zl_ref_xl": 0.7228, "zl_ref_yl":0.6911, "zl_ref_zl":-0.0055}}}

    # Params
    T = 298 # K

    #force_constants = list(boresch_dict["force_constants"].values()) # kcal mol-1 A-2 or rad-2
    #prod_force_constants = np.prod(force_constants)

    prefactor = 8*(pi**2)*v0 # Divide this to account for force constants of 0
    force_constants = []
    k_theta = 0
    

    # Loop through and correct for angle force constants of zero,
    # which break the analytical correction
    for k, val in cart_dict["force_constants"].items():
        if val == 0:
            if k[3] == "r":
                print("Error: Positional restraints must not be zero")
                sys.exit(-1)
            if k in ["k_phi", "k_psi"]:
                prefactor /= 2*pi
            if k == "k_theta":
                prefactor /= 2
                k_theta = val
        else:
            force_constants.append(val)

    n_nonzero_k = len(force_constants)
    prod_force_constants = np.prod(force_constants)

    # Calculation
    numerator = np.sqrt(prod_force_constants)
    denominator_1 = ((2*pi*R*T)**(n_nonzero_k/2))/pi**((bool(k_theta))/2) # If one of the force consts 
                                                                          # is k_theta, divide by sqrt(pi)
    if bool(k_theta):
        denominator_2 = dawsn(((R*T)**(0.5))/((2*k_theta)**0.5))
    else:
        denominator_2 = 1

    dg = -R*T*log((prefactor*numerator)/(denominator_1 * denominator_2))

    print(f"Analytical correction for releasing Cartesian restraints = {dg:.2f} kcal mol-1")

Analytical correction for releasing Cartesian restraints = -8.86 kcal mol-1
